In [1]:
# Uncomment these lines when running for the first time to install the required packages.  This is mostly required for local development.
# # %pip install openai
# %pip install google-auth
# %pip install google-cloud-bigquery
# %pip install google-cloud-storage
# %pip install lca
# %pip install banjo
# %pip install tqdm
# %pip install oauth2client
# %pip install tqdm.contrib
# %pip install google-generativeai
# %pip install google-genai
# %pip install av

In [2]:
import os
import time
import tempfile
import logging
from datetime import datetime, timedelta, timezone
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

import requests
import pandas as pd
import av
from tqdm import tqdm
from tqdm.contrib.concurrent import thread_map
from functools import partial

from google.cloud import bigquery, storage
from google import genai
from google.api_core.exceptions import TooManyRequests

from banjo import utils
from banjo.utils.shibainu import (
    Classification,
    estimate_run_cost,
    configure_logger,
)

# Import maps viral places classification utils
from utils.constant import get_viral_places_query, place_agg_dict, PLACE_REQUIRED_KEYS, PLACE_LIST_COLS, VIDEO_REQUIRED_KEYS, SELECTED_COLS, STORY_COLS_RENAME
from utils.helper import download_and_upload, topk_by_score_per_place, parse_incident_json_broken, parse_incident_safe, majority_vote, combine_text_list
from utils.prompt import VIDEO_CLASSIFIER_PROMPT, TEXT_CLASSIFIER_PROMPT

# Configure logging
configure_logger(level=logging.ERROR)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Logger shibainu (ERROR)>

#### Parameters

In [3]:
START_DATE = '20250703'
END_DATE = '20250705'
VIEW_WEIGHT = 0.6
FRESHNESS_WEIGHT = 0.4
SAMPLE_PER_PLACE = 5
MAX_WORKERS = 10

# Initialize GCS client
client = storage.Client(project="myaigcp")
BUCKET_NAME = "shiba-inu-temp"
BUCKET_FOLDER = "maps_events_20250720"
BUCKET = client.bucket(BUCKET_NAME)

# Save Destination 
WRITE_PROJECT_ID = "sc-bq-gcs-billingonly"
WRITE_DATASET = "temp_datascience"
WRITE_TABLE_NAME = "maps_viral_places_classification"

DESTINATION = f"{WRITE_PROJECT_ID}.{WRITE_DATASET}.{WRITE_TABLE_NAME}"
# SERVICE_ACCOUNT = 'shiba-inu@sc-product-datascience.iam.gserviceaccount.com'

#### Import Data and Upload Media URL to GCS 

In [4]:
#Step 1: Import Data
query = get_viral_places_query(START_DATE, END_DATE, VIEW_WEIGHT, FRESHNESS_WEIGHT)
df = utils.gbq.read_gbq(query, 
                    project_id="myaigcp",
                    dialect="standard",
                    priority="interactive")


E0000 00:00:1762470515.783535 1692827 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [5]:
# Step 2: Download and Upload Media URL to GCS
worker = partial(
    download_and_upload,
    bucket=BUCKET,
    bucket_name=BUCKET_NAME,
    bucket_folder=BUCKET_FOLDER,
    url_col='media_url',
    id_col='story_snap_id',
)
rows = df.to_dict(orient="records")       
with ThreadPoolExecutor(max_workers=10) as ex:
    results = list(tqdm(ex.map(worker, rows), total=len(rows)))

# Write the results back to the original df
df.loc[df.index, "gcs_url"] = results


  1%|          | 11/2096 [00:02<05:10,  6.72it/s] 

Error processing https://cf-st.sc-cdn.net/d/ozXhUb7dcBDhufHmaObo9.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCKjeocMGSAJQSWABogE3CIoIEiUKIwiR004gATDgAzjWBkABSg4KCUxBSFBLTUlAOhD0A1D8TWgCIgsSACoHSVJaWFNPWZAD_E0%3D&uc=73 for story_id W7_EDlXWTBiXAEEniNoMPwAAYZmhyZ2NtaGZlAZfX8aK2AZfX8aHdAAAAAQ: 404 Client Error: Not Found for url: https://cf-st.sc-cdn.net/d/ozXhUb7dcBDhufHmaObo9.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCKjeocMGSAJQSWABogE3CIoIEiUKIwiR004gATDgAzjWBkABSg4KCUxBSFBLTUlAOhD0A1D8TWgCIgsSACoHSVJaWFNPWZAD_E0%3D&uc=73
Error processing https://cf-st.sc-cdn.net/d/3S9RVR8l0rofUGAnyiogd.1034.IRZXSOY?mo=GlUaFDIBBDoBfUIGCNTJoMMGSAJQSWABogE4CIoIEiYKJAjuvDwgATDgAzjWBkABSg8KCq4BR0RGOjM_MDgQ9ANQyjloAiILEgAqB0lSWlhTT1mQA8o5&uc=73 for story_id W7_EDlXWTBiXAEEniNoMPwAAYY3Z6enBsbXluAZfWz1yJAZfWz1v4AAAAAQ: 404 Client Error: Not Found for url: https://cf-st.sc-cdn.net/d/3S9RVR8l0rofUGAnyiogd.1034.IRZXSOY?mo=GlUaFDIBBDoBfUIGCNTJoMMGSAJQSWABogE4CIoIEiYKJAjuvDwgATDgAzjWBkABSg8KCq4BR0RGOjM_MDgQ9ANQyjloAiILEgAqB0lSWlhTT1m

  7%|▋         | 145/2096 [00:12<04:13,  7.68it/s]

Error processing https://cf-st.sc-cdn.net/d/RwgXDFyGZMAjFpBOsNPvA.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCOzvo8MGSAJQSWABogE3CIoIEiUKIwjYtR4gATDgAzjUBkABSg4KCTwdHB8bHBsZFBD0A1CmRWgCIgsSACoHSVJaWFNPWZADpkU%3D&uc=73 for story_id W7_EDlXWTBiXAEEniNoMPwAAYb2F5dmlnZ2l2AZfaB_fvAZfaB_fXAAAAAw: 404 Client Error: Not Found for url: https://cf-st.sc-cdn.net/d/RwgXDFyGZMAjFpBOsNPvA.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCOzvo8MGSAJQSWABogE3CIoIEiUKIwjYtR4gATDgAzjUBkABSg4KCTwdHB8bHBsZFBD0A1CmRWgCIgsSACoHSVJaWFNPWZADpkU%3D&uc=73
Error processing https://cf-st.sc-cdn.net/d/CFKyBfxC6UEz2q2hsLE2X.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCOrvo8MGSAJQSWABogE3CIoIEiUKIwjAxCUgATDgAzjUBkABSg4KCTcSGhwdHx4eIhD0A1CuTmgCIgsSACoHSVJaWFNPWZADrk4%3D&uc=73 for story_id W7_EDlXWTBiXAEEniNoMPwAAYd3huZnZ6dnpvAZfaB_GLAZfaB_FyAAAAAw: 404 Client Error: Not Found for url: https://cf-st.sc-cdn.net/d/CFKyBfxC6UEz2q2hsLE2X.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCOrvo8MGSAJQSWABogE3CIoIEiUKIwjAxCUgATDgAzjUBkABSg4KCTcSGhwdHx4eIhD0A1CuTmgCIgsSACoHSVJaWFNPW

 14%|█▍        | 302/2096 [00:21<02:29, 12.03it/s]

...

 93%|█████████▎| 1940/2096 [02:08<00:08, 18.83it/s]

Error processing https://cf-st.sc-cdn.net/d/Znzu32GrWxWnRcwIqmKjR.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCLPsn8MGSAJQSWABogE3CIoIEiUKIwiXkzQgATDgAzjWBkABSg4KCUg7NjApQTA4MxD0A1D2O2gCIgsSACoHSVJaWFNPWZAD9js%3D&uc=73 for story_id W7_EDlXWTBiXAEEniNoMPwAAYYXl6eWppZmpuAZfWGTePAZfV-4rgAAAAAQ: 404 Client Error: Not Found for url: https://cf-st.sc-cdn.net/d/Znzu32GrWxWnRcwIqmKjR.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCLPsn8MGSAJQSWABogE3CIoIEiUKIwiXkzQgATDgAzjWBkABSg4KCUg7NjApQTA4MxD0A1D2O2gCIgsSACoHSVJaWFNPWZAD9js%3D&uc=73


 98%|█████████▊| 2064/2096 [02:15<00:01, 18.39it/s]

Error processing https://cf-st.sc-cdn.net/d/OOVDJYPVETqMbf9R2C1X6.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCJmtlMMGSAJQSWABogE3CIoIEiUKIwi23zcgATDgAzjUBkABSg4KCW0XUyAlJhwxRBD0A1DyTWgCIgsSACoHSVJaWFNPWZAD8k0%3D&uc=73 for story_id W7_EDlXWTBiXAEEniNoMPwAAYd2x1cmJmYW9vAZfK37m_AZfK3QseAAAAAQ: 404 Client Error: Not Found for url: https://cf-st.sc-cdn.net/d/OOVDJYPVETqMbf9R2C1X6.1034.IRZXSOY?mo=GlQaFDIBBDoBfUIGCJmtlMMGSAJQSWABogE3CIoIEiUKIwi23zcgATDgAzjUBkABSg4KCW0XUyAlJhwxRBD0A1DyTWgCIgsSACoHSVJaWFNPWZAD8k0%3D&uc=73


100%|██████████| 2096/2096 [02:16<00:00, 15.32it/s]


#### Call Shibainu Classification

In [6]:
# Step 1: Initialize Shibainu Classification
video_classifier = Classification(
    provider_name="gemini",
    model_name='gemini-2.0-flash',
    input_type="video",
    provider_config={"project_id": "myaigcp",
                    "location": "us-central1"}, #this config is used for gemini only
    processor_config = {
        "processing_mode": "image_url",  # "image_url" or "bytes"
        'return_direct_url': True       
    },
    model_parameters={
        "temperature": 0,
        "max_token": 1024
    },
    prompt=VIDEO_CLASSIFIER_PROMPT
)

video_classifier.get_result(video_classifier.send_message('What is your model?'))

'I am a large language model, trained by Google.'

In [7]:
# Step2: Choose top K videos per place and run classification
df_selected_raw = topk_by_score_per_place(df, group_col='place_id', order_col='score', filter_col='gcs_url', k=SAMPLE_PER_PLACE)
video_classifier_results = thread_map(video_classifier.classify, df_selected_raw['gcs_url'].tolist(), max_workers=MAX_WORKERS)

 82%|████████▏ | 40/49 [00:30<00:05,  1.73it/s]

2025-11-06 15:11:40,968 - shibainu - ERROR - Error during classification after 21.37 seconds: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}
2025-11-06 15:11:41,018 - shibainu - ERROR - Error during classification after 20.68 seconds: Server disconnected without sending a response.
2025-11-06 15:11:41,804 - shibainu - ERROR - Error during classification after 22.20 seconds: Cannot send a request, as the client has been closed.


100%|██████████| 49/49 [00:53<00:00,  1.09s/it]


In [8]:
# Step 3: Parse labels and construct the summary table
video_labels = [video_classifier.get_result(r) for r in video_classifier_results]
video_usage  = [video_classifier.get_token_usage(r) for r in video_classifier_results]

df_video_summary = df_selected_raw.copy()
df_video_summary.loc[:, "video_labels"] = video_labels
df_video_summary.loc[:, "video_prompt_tokens"]= [u.get("prompt_tokens") for u in video_usage]
df_video_summary.loc[:, "video_completion_tokens"]= [u.get("completion_tokens") for u in video_usage]

# 3.1 Parse labels
video_parsed = (
    df_video_summary["video_labels"]
    .map(lambda x: parse_incident_safe(x, VIDEO_REQUIRED_KEYS))
    .apply(pd.Series)
)

# 3.2 Attach parsed columns
df_video_summary = pd.concat([df_video_summary, video_parsed], axis=1)

#### Using Text Classification to Consolidate

In [9]:
# Group and aggregate
df_place_raw = df_video_summary.groupby('place_id', as_index=False).agg(place_agg_dict)

In [10]:
# Text classification
text_classifier = Classification(
    provider_name="gemini",
    model_name='gemini-2.0-flash',
    input_type="text",
    provider_config={"project_id": "myaigcp",
                    "location": "us-central1"}, #this config is used for gemini only
    # processor_config = {
    #     "processing_mode": "image_url",  # "image_url" or "bytes"
    #     'return_direct_url': True       
    # },
    model_parameters={
        "temperature": 0,
        "max_token": 2000,
    },
    prompt=TEXT_CLASSIFIER_PROMPT
)

text_results = thread_map(text_classifier.classify, df_place_raw.to_dict("records"), max_workers=10)

100%|██████████| 11/11 [00:04<00:00,  2.21it/s]


In [ ]:
# Construct the results
text_labels = [text_classifier.get_result(r) for r in text_results]

df_place_summary = df_place_raw.copy()
df_place_summary.drop(columns=["key_objects", "activity_type", "contributing_context",
                               "short_description", "long_description", "keywords"], inplace=True)
df_place_summary.loc[:, "text_labels"] = text_labels


# Parse labels
text_parsed = (
    df_place_summary["text_labels"]
    .map(lambda x: parse_incident_safe(x, PLACE_REQUIRED_KEYS))
    .apply(pd.Series)
)

df_place_summary = pd.concat([df_place_summary, text_parsed], axis=1)

#### Consolidate the Output

In [12]:
df_video_summary = df_video_summary.rename(
    columns={col: f"video_{col}" for col in STORY_COLS_RENAME}
)
df_merged = df_place_summary.merge(df_video_summary, on=['place_id', 'continent','place_name', 'place_country_code',
                                                        'detection_start_time', 'detection_end_time'], how='left')

#### Write Output to BQ

In [13]:
df_merged.to_gbq(destination_table= DESTINATION, project_id= WRITE_PROJECT_ID, if_exists='replace')
print(f"`{DESTINATION}` was created successfully!")

100%|██████████| 1/1 [00:00<00:00, 702.68it/s]

`sc-bq-gcs-billingonly.temp_datascience.maps_viral_places_classification` was created successfully!


In [15]:
df_merged

,place_id,continent,place_name,place_country_code,detection_start_time,detection_end_time,virality_potential,event_type,event_scale,event_duration,...,video_keywords,video_event_type,video_event_scale,video_event_duration,video_event_intensity,video_associated_mood,video_key_objects_entities,video_activity_type,video_contributing_context,video_virality_potential
0,4c41d4e0-a681-11e8-9074-7f02ac589aae,Europe,Marmande,FR,2025-07-05 06:30:00,2025-07-05 15:45:00,0.80,Social Gathering,Regional,Short-term,...,"Concert, Music, Dj, Crowd, Lights, Stage, Nigh...",Social Gathering,Regional,Short-term,High,Positive,"Crowd, Stage, Lights, Djs","Dancing, Performing, Listening","Night, Indoor",1.0
1,4c41d4e0-a681-11e8-9074-7f02ac589aae,Europe,Marmande,FR,2025-07-05 06:30:00,2025-07-05 15:45:00,0.80,Social Gathering,Regional,Short-term,...,"Concert, Live performance, Stage, Crowd, Music...",Social Gathering,Regional,Short-term,High,Positive,"Stage, Crowd, Lights","Performing, Listening, Cheering","Night, Indoor",1.0
2,4c41d4e0-a681-11e8-9074-7f02ac589aae,Europe,Marmande,FR,2025-07-05 06:30:00,2025-07-05 15:45:00,0.80,Social Gathering,Regional,Short-term,...,"Concert, Music, Crowd, Lights, Stage, Performa...",Social Gathering,Regional,Short-term,High,Positive,"Crowd, Stage, Lights","Cheering, Listening, Dancing","Night, Outdoor",1.0
3,4c41d4e0-a681-11e8-9074-7f02ac589aae,Europe,Marmande,FR,2025-07-05 06:30:00,2025-07-05 15:45:00,0.80,Social Gathering,Regional,Short-term,...,"Music festival, Crowd, Dj, Stage, Ctrl+b, Outd...",Social Gathering,Regional,Short-term,High,Positive,"Crowd, Stage, Dj","Dancing, Listening, Partying","Outdoor, Sunny, Daytime",1.0
4,4c41d4e0-a681-11e8-9074-7f02ac589aae,Europe,Marmande,FR,2025-07-05 06:30:00,2025-07-05 15:45:00,0.80,Social Gathering,Regional,Short-term,...,"Garorock festival, Crowd, Music festival, Coca...",Social Gathering,Regional,Short-term,Medium,Positive,"Crowd, Umbrellas","Relaxing, Walking, Socializing","Sunny day, Outdoor",0.0
5,51c10b20-a681-11e8-8153-d333dde2706a,Asia,Kerbela,IQ,2025-07-05 04:00:00,2025-07-05 15:45:00,0.00,Social Gathering,Local,Short-term,...,"Religious leader, Speech, Sermon, Turban, Micr...",Social Gathering,Local,Short-term,Medium,Negative,"Religious leader, Microphone, People","Speaking, Listening","Indoor, Religious setting",0.0
6,51c10b20-a681-11e8-8153-d333dde2706a,Asia,Kerbela,IQ,2025-07-05 04:00:00,2025-07-05 15:45:00,0.00,Social Gathering,Local,Short-term,...,"Cooking, Pots, Outdoors, Palm trees, Food prep...",Social Gathering,Local,Short-term,Medium,Neutral,"Pots, People, Food","Cooking, Preparing food","Daytime, Outdoor",0.0
7,51c10b20-a681-11e8-8153-d333dde2706a,Asia,Kerbela,IQ,2025-07-05 04:00:00,2025-07-05 15:45:00,0.00,Social Gathering,Local,Short-term,...,"Man, Sunglasses, Car, Music, Driving, Sunroof,...",Miscellaneous or Other,Local,Short-term,Low,Neutral,"Man, Car, Sunglasses","Listening, Driving","Daytime, Sunny",0.0
8,51c10b20-a681-11e8-8153-d333dde2706a,Asia,Kerbela,IQ,2025-07-05 04:00:00,2025-07-05 15:45:00,0.00,Social Gathering,Local,Short-term,...,"Men, Walking, Oil, Supplies, Group, Sidewalk",Human Activity (Non-Social),Local,Short-term,Low,Neutral,"Men, Oil bottle, Box, Bags","Walking, Carrying","Daytime, Sidewalk",0.0
9,51c10b20-a681-11e8-8153-d333dde2706a,Asia,Kerbela,IQ,2025-07-05 04:00:00,2025-07-05 15:45:00,0.00,Social Gathering,Local,Short-term,...,"Driving, Car, Music, Sunglasses, Polo shirt, S...",Transportation Related,Local,Short-term,Low,Neutral,"Car, Man, Sunglasses","Driving, Listening","Daytime, Sunny",0.0
